<a href="https://colab.research.google.com/github/leonorsilva/projecto/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [267]:
!pip install transformers

In [268]:
import torch
import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [405]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        #self.pos_embedding = nn.Embedding(max_length, hid_dim)  #position embedding

        self.layers = nn.ModuleList([EncoderLayer(hid_dim,
                                                  n_heads,
                                                  pf_dim,
                                                  dropout,
                                                  device)
                                     for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, src, src_mask):

        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        #pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        #pos = [batch size, src len]

        src = self.dropout((self.tok_embedding(src) * self.scale)) #+ self.pos_embedding(pos))

        #src = [batch size, src len, hid dim]

        for layer in self.layers:

            src = layer(src, src_mask)

        #src = [batch size, src len, hid dim]
        output = self.fc_out(src)
        output = torch.softmax(torch.mean(output, 1),1)

        return output



In [270]:
class EncoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):

        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len]

        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)

        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        #src = [batch size, src len, hid dim]

        #positionwise feedforward
        _src = self.positionwise_feedforward(src)

        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))

        #src = [batch size, src len, hid dim]

        return src

In [415]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()

        assert hid_dim % n_heads == 0

        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)


        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]

        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        #energy = [batch size, n heads, query len, key len]
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        attention = torch.softmax(energy, dim = -1)

        #attention = [batch size, n heads, query len, key len]

        x = torch.matmul(self.dropout(attention), V)

        #x = [batch size, n heads, query len, head dim]

        x = x.permute(0, 2, 1, 3).contiguous()

        #x = [batch size, query len, n heads, head dim]

        x = x.view(batch_size, -1, self.hid_dim)

        #x = [batch size, query len, hid dim]

        x = self.fc_o(x)

        #x = [batch size, query len, hid dim]

        return x, attention

In [416]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        #x = [batch size, seq len, hid dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        #x = [batch size, seq len, pf dim]

        x = self.fc_2(x)

        #x = [batch size, seq len, hid dim]

        return x

In [273]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [274]:
INPUT_DIM = 15000
OUTPUT_DIM = 5
HID_DIM = 4
ENC_LAYERS = 1
ENC_HEADS = 2
ENC_PF_DIM = 3
ENC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM,
              OUTPUT_DIM,
              HID_DIM,
              ENC_LAYERS,
              ENC_HEADS,
              ENC_PF_DIM,
              ENC_DROPOUT,
              device)

In [303]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
token= tokenizer.encode_plus('I am awesome', max_length=5, truncation=True,
                                   padding='max_length', add_special_tokens=True,
                                   return_tensors='tf')

In [304]:
input=torch.Tensor(token['input_ids'].numpy())
input=input.type(torch.int32)

mask=torch.Tensor(token['attention_mask'].numpy())
mask=mask.type(torch.int32)

In [305]:
input2=torch.Tensor([token['input_ids'].numpy()[0],token['input_ids'].numpy()[0]])
input2=input2.type(torch.int32)


mask2=torch.Tensor([token['attention_mask'].numpy()[0],token['attention_mask'].numpy()[0]])
mask2=mask2.type(torch.int32)

In [306]:
input

tensor([[  101,   146,  1821, 14918,   102]], dtype=torch.int32)

In [307]:
mask

tensor([[1, 1, 1, 1, 1]], dtype=torch.int32)

In [278]:
mask=(input != mask).unsqueeze(1).unsqueeze(2)

In [279]:
enc(input,mask)

tensor([[[-2.1532, -2.7696,  3.6205, -1.7788],
         [ 1.4905, -0.8644, -1.6782, -1.1402],
         [-0.6956,  0.4498,  0.7536, -0.7469],
         [-3.3077,  1.5870,  2.0204, -1.3027],
         [-2.1564, -1.3723,  3.0467, -1.3844]]], grad_fn=<ViewBackward0>)


tensor([[0.1488, 0.1200, 0.2552, 0.1718, 0.3043]], grad_fn=<SoftmaxBackward0>)

In [280]:
mask2=(input2 != mask2).unsqueeze(1).unsqueeze(2)

In [281]:
mask2

tensor([[[[True, True, True, True, True]]],


        [[[True, True, True, True, True]]]])

In [282]:
enc(input2,mask2)

tensor([[[-2.8837, -1.7239,  4.1456, -2.0590],
         [ 1.4905, -0.8644, -1.6782, -1.1402],
         [ 0.1102,  0.2170,  0.7109, -0.5322],
         [-3.3077,  1.5870,  2.0204, -1.3027],
         [-1.0670, -1.1993,  1.7953, -1.9049]],

        [[-2.8837, -1.7239,  4.1456, -2.0590],
         [ 0.9257,  0.5291, -1.5961, -0.9235],
         [ 0.2320,  0.2364,  0.5709, -0.5904],
         [-3.3077,  1.5870,  2.0204, -1.3027],
         [-2.1564, -1.3723,  3.0467, -1.3844]]], grad_fn=<ViewBackward0>)


tensor([[0.1595, 0.1229, 0.2377, 0.1966, 0.2833],
        [0.1580, 0.1214, 0.2453, 0.1839, 0.2913]], grad_fn=<SoftmaxBackward0>)

In [422]:
import pandas as pd

train=pd.DataFrame(pd.read_csv('train.tsv',sep='\t'))

In [423]:
train['tokens']=None
train['mask']=None

In [424]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(train['Phrase']):
    tokens = tokenizer.encode_plus(phrase, max_length=50, truncation=True,
                                   padding='max_length', add_special_tokens=True,
                                   return_tensors='tf')
    # assign tokenized outputs to respective rows in numpy arrays
    train['tokens'][i]=tokens['input_ids'].numpy()[0]
    train['mask'][i]=tokens['attention_mask'].numpy()[0]


<ipython-input-424-90410cdc0ed0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['tokens'][i]=tokens['input_ids'].numpy()[0]
<ipython-input-424-90410cdc0ed0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['mask'][i]=tokens['attention_mask'].numpy()[0]


In [425]:
train=pd.get_dummies(train, columns =['Sentiment'])

In [426]:
train

,PhraseId,SentenceId,Phrase,tokens,mask,Sentiment_0,Sentiment_1,Sentiment_2,Sentiment_3,Sentiment_4
0,1,1,A series of escapades demonstrating the adage ...,"[101, 138, 1326, 1104, 13936, 25265, 16913, 15...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,1,0,0,0
1,2,1,A series of escapades demonstrating the adage ...,"[101, 138, 1326, 1104, 13936, 25265, 16913, 15...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0,1,0,0
2,3,1,A series,"[101, 138, 1326, 102, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0
3,4,1,A,"[101, 138, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0
4,5,1,series,"[101, 1326, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
156055,156056,8544,Hearst 's,"[101, 22038, 2050, 112, 188, 102, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0
156056,156057,8544,forced avuncular chortles,"[101, 2257, 170, 25247, 26405, 5552, 22572, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",0,1,0,0,0
156057,156058,8544,avuncular chortles,"[101, 170, 25247, 26405, 5552, 22572, 12148, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...",0,0,0,1,0
156058,156059,8544,avuncular,"[101, 170, 25247, 26405, 5552, 102, 0, 0, 0, 0...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0


In [410]:
input=torch.Tensor([train['tokens'][0]])
mask=torch.Tensor([train['mask'][0]])

input=input.type(torch.int32)
mask=mask.type(torch.int32)

In [373]:
mask=(input != mask).unsqueeze(1).unsqueeze(2)

In [427]:
INPUT_DIM = 30000
OUTPUT_DIM = 5
HID_DIM = 512
ENC_LAYERS = 1
ENC_HEADS = 2
ENC_PF_DIM = 3
ENC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM,
              OUTPUT_DIM,
              HID_DIM,
              ENC_LAYERS,
              ENC_HEADS,
              ENC_PF_DIM,
              ENC_DROPOUT,
              device)

In [411]:
enc(input,mask)

tensor([[[-11.7663, -16.3809, -18.6960,  ...,   9.2368, -22.2358,   6.8398],
         [ -1.6651,  12.0563, -19.6013,  ...,  10.2947, -10.2525,   5.8467],
         [-10.1870,  -6.0070, -10.1302,  ...,  -6.1951,  12.5548, -12.1602],
         ...,
         [-12.4946, -10.9464,  -1.0392,  ..., -11.8919, -12.3754,  -5.5766],
         [ -4.1557,   2.3170,   5.2282,  ..., -17.6023, -16.1070,  -4.2865],
         [ -9.4427,  -8.5612,   9.4880,  ..., -19.9716, -12.0389,  -7.7614]]],
       grad_fn=<ViewBackward0>)
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0]], dtype=torch.int32)


tensor([[0.1910, 0.2316, 0.2076, 0.2070, 0.1629]], grad_fn=<SoftmaxBackward0>)

In [418]:
data=train.sample(4)  #with repetition

inputs= torch.Tensor(np.vstack(data['tokens']))
masks=torch.Tensor(np.vstack(data['mask']))

inputs=inputs.type(torch.int32)
masks=masks.type(torch.int32)

masks=(inputs != masks).unsqueeze(1).unsqueeze(2)

In [419]:
enc(inputs,masks)

tensor([[0.1578, 0.2619, 0.1794, 0.1574, 0.2436],
        [0.1808, 0.2649, 0.1200, 0.1444, 0.2899],
        [0.1702, 0.2615, 0.1098, 0.1205, 0.3380],
        [0.1745, 0.2697, 0.1212, 0.1396, 0.2950]], grad_fn=<SoftmaxBackward0>)

In [438]:
import torch.optim as optim

optimizer = optim.Adam(enc.parameters(), lr=0.05)
loss_fn= nn.CrossEntropyLoss()

def trainning(epoch):
    running_loss = 0.
    last_loss = 0.
    batch=64

    for i in range(epoch):
        # Every data instance is an input + label pair
        data=train.sample(batch)  #with repetition

        inputs= torch.Tensor(np.vstack(data['tokens']))
        masks=torch.Tensor(np.vstack(data['mask']))

        inputs=inputs.type(torch.int32)
        masks=masks.type(torch.int32)

        masks=(inputs != masks).unsqueeze(1).unsqueeze(2)

        labels=torch.Tensor(data[['Sentiment_0','Sentiment_1','Sentiment_2','Sentiment_3','Sentiment_4']].to_numpy())
        labels=labels.type(torch.int32)
        print(labels)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = enc(inputs,masks)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()



    return last_loss

trainning(5)

tensor([[0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0],


RuntimeError: ignored

In [436]:
data[['Sentiment_0','Sentiment_1','Sentiment_2','Sentiment_3','Sentiment_4']].to_numpy()

array([[0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]], dtype=uint8)

In [432]:
torch.Tensor(data['Sentiment_0','Sentiment_1','Sentiment_2','Sentiment_3','Sentiment_4'])

KeyError: ignored